<img src="https://microsoft.github.io/Accera/assets/Accera_darktext.png" alt="Accera logo" width="600"/>

# Accera Quickstart

### Install Accera

First, we'll install Accera using `pip`.

#### Additional Requirements
This notebook is launched in an environment that has `gcc` pre-installed. If you are running this notebook locally, ensure that you run `apt install gcc` as well.

In [6]:
!pip3 install accera==1.2.1.dev27

     |████████████████████████████████| 30.0 MB 17.7 MB/s eta 0:00:01
  Attempting uninstall: accera
    Found existing installation: accera 1.2.0
    Uninstalling accera-1.2.0:
      Successfully uninstalled accera-1.2.0


### Build your first package

We'll build a package called `"mypackage"`, containing one function called `"hello_accera"`.

This function performs the operation `B += A` on arrays `A` and `B`.

In [14]:
import accera as acc
import hatlib as hat
import numpy as np

A = acc.Array(role=acc.Array.Role.INPUT, shape=(16, 16))
B = acc.Array(role=acc.Array.Role.INPUT_OUTPUT, shape=(16, 16))

nest = acc.Nest(shape=(16, 16))
i, j = nest.get_indices()

@nest.iteration_logic
def _():
    B[i, j] += A[i, j]

package = acc.Package()
function = package.add(nest, args=(A, B), base_name="hello_accera")
package.build(name="mypackage")

hat.create_dynamic_package("mypackage.hat", "mypackage_dyn.hat")

Building function hello_accera_9e9c7806
mkdir /root/accera-demo/_tmp/logs
mkdir /root/accera-demo/_tmp/AcceraGPUUtilities_intermediate
mkdir /root/accera-demo/_tmp
mkdir /root/accera-demo/_tmp/generator
mkdir /root/accera-demo/_tmp/lib
mkdir /root/accera-demo/_tmp/AcceraGPUUtilities

cd /root/accera-demo/_tmp/AcceraGPUUtilities
"/usr/local/lib/python3.8/dist-packages/accera/bin/acc-opt" --verify-each=false --acc-to-llvm="dump-passes=false dump-intra-pass-ir=false target=host enable-profiling=false" "/root/accera-demo/_tmp/AcceraGPUUtilities.mlir"


cd /root/accera-demo/_tmp/AcceraGPUUtilities_intermediate
"/usr/local/lib/python3.8/dist-packages/accera/bin/mlir-translate" --mlir-print-op-on-diagnostic --mlir-to-llvmir -o="/root/accera-demo/_tmp/AcceraGPUUtilities/AcceraGPUUtilities.ll" "/root/accera-demo/_tmp/AcceraGPUUtilities/AcceraGPUUtilities_llvm.mlir"


cd /root/accera-demo/_tmp/AcceraGPUUtilities_intermediate
"/usr/local/lib/python3.8/dist-packages/accera/bin/opt" -O3 -fp-contrac

### Load the package and call the function

After the package is built, this produces a dynamic library (`mypackage.so`). The code below calls the `hello_accera` function we just built with two example NumPy arrays.

In [ ]:
hat_package = hat.load("mypackage_dyn.hat")
hello_accera = hat_package["hello_accera"]

# call the function with test input
A_test = np.ones((16, 16), dtype=np.float32)
B_test = np.ones((16, 16), dtype=np.float32) * 2.14
print(f"Before hello_accera(): {B_test}")
hello_accera(A_test, B_test)
print(f"After hello_accera(): {B_test}")